In [1]:
from hashlib import sha3_256

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Extract

In [2]:
df = pd.read_json("../data/scraped/tps.jsonl", lines=True, convert_dates=["ts"])
df.head()

,chart,images,administrasi,psu,ts,status_suara,status_adm,url,mode
0,None,[https://sirekap-obj-formc.kpu.go.id/93b4/pemi...,None,None,2024-02-16 08:00:28,False,False,https://sirekap-obj-data.kpu.go.id/pemilu/hhcw...,NaN
1,"{'null': None, '100025': 44, '100026': 66, '10...",[https://sirekap-obj-formc.kpu.go.id/d167/pemi...,"{'suara_sah': 164, 'suara_total': 170, 'pemili...",None,2024-02-17 16:52:47,True,True,https://sirekap-obj-data.kpu.go.id/pemilu/hhcw...,hhcw
2,None,"[None, None, None]",None,None,2024-02-15 07:00:44,False,False,https://sirekap-obj-data.kpu.go.id/pemilu/hhcw...,NaN
3,"{'null': None, '100025': 104, '100026': 96, '1...","[None, https://sirekap-obj-formc.kpu.go.id/4aa...",None,None,2024-02-25 14:00:00,True,False,https://sirekap-obj-data.kpu.go.id/pemilu/hhcw...,hhcw
4,None,"[None, None, None]",None,None,2024-02-15 07:00:44,False,False,https://sirekap-obj-data.kpu.go.id/pemilu/hhcw...,NaN


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 823378 entries, 0 to 823377
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   chart         647735 non-null  object        
 1   images        823378 non-null  object        
 2   administrasi  529561 non-null  object        
 3   psu           2229 non-null    object        
 4   ts            823378 non-null  datetime64[ns]
 5   status_suara  823378 non-null  bool          
 6   status_adm    823378 non-null  bool          
 7   url           823378 non-null  object        
 8   mode          454910 non-null  object        
dtypes: bool(2), datetime64[ns](1), object(6)
memory usage: 45.5+ MB


# Transform

In [4]:
PASLON_NAMES = {
    "100025": "H. ANIES RASYID BASWEDAN, Ph.D. - Dr. (H.C.) H. A. MUHAIMIN ISKANDAR",
    "100026": "H. PRABOWO SUBIANTO - GIBRAN RAKABUMING RAKA",
    "100027": "H. GANJAR PRANOWO, S.H., M.I.P. - Prof. Dr. H. M. MAHFUD MD",
}

In [5]:
def hash_url(s: str) -> str:
    return sha3_256(s.encode("utf-8")).hexdigest()

In [6]:
df_stats = (
    pd.json_normalize(df["chart"])
    .drop(columns=["null"])
    .rename(columns={k: f"suara_paslon_{i}" for i,k in enumerate(PASLON_NAMES.keys())})
)

df_stats.head()

,suara_paslon_0,suara_paslon_1,suara_paslon_2
0,NaN,NaN,NaN
1,44.0,66.0,54.0
2,NaN,NaN,NaN
3,104.0,96.0,49.0
4,NaN,NaN,NaN


In [7]:
df_admin = pd.json_normalize(df["administrasi"])
df_admin.head()

,suara_sah,suara_total,pemilih_dpt_j,pemilih_dpt_l,pemilih_dpt_p,pengguna_dpt_j,pengguna_dpt_l,pengguna_dpt_p,pengguna_dptb_j,pengguna_dptb_l,pengguna_dptb_p,suara_tidak_sah,pengguna_total_j,pengguna_total_l,pengguna_total_p,pengguna_non_dpt_j,pengguna_non_dpt_l,pengguna_non_dpt_p
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,164.0,170.0,186.0,89.0,97.0,162.0,76.0,86.0,8.0,1.0,7.0,6.0,170.0,77.0,93.0,0.0,0.0,0.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df_pages = pd.DataFrame(df["images"].tolist(), columns=[f"scanned_page_{i}" for i in range(1, 4)], index=df.index)
df_pages.head()

,scanned_page_1,scanned_page_2,scanned_page_3
0,https://sirekap-obj-formc.kpu.go.id/93b4/pemil...,None,None
1,https://sirekap-obj-formc.kpu.go.id/d167/pemil...,https://sirekap-obj-formc.kpu.go.id/d167/pemil...,https://sirekap-obj-formc.kpu.go.id/d167/pemil...
2,None,None,None
3,None,https://sirekap-obj-formc.kpu.go.id/4aa9/pemil...,https://sirekap-obj-formc.kpu.go.id/4aa9/pemil...
4,None,None,None


In [9]:
df_final = (
    df[["ts", "mode", "psu", "status_suara", "status_adm", "url"]]
    .copy()
    .assign(
        id=df["url"].apply(hash_url),
        kode=df["url"].str.extract(r"(\d+)\.json"),
    )
    .join(df_stats)
    .join(df_admin)
    .join(df_pages)
)

df_final.head()

,ts,mode,psu,status_suara,status_adm,url,id,kode,suara_paslon_0,suara_paslon_1,...,suara_tidak_sah,pengguna_total_j,pengguna_total_l,pengguna_total_p,pengguna_non_dpt_j,pengguna_non_dpt_l,pengguna_non_dpt_p,scanned_page_1,scanned_page_2,scanned_page_3
0,2024-02-16 08:00:28,NaN,None,False,False,https://sirekap-obj-data.kpu.go.id/pemilu/hhcw...,57b06ebd3928fa64149945243adfd8d231ffc71f26ea22...,3471131004004,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://sirekap-obj-formc.kpu.go.id/93b4/pemil...,None,None
1,2024-02-17 16:52:47,hhcw,None,True,True,https://sirekap-obj-data.kpu.go.id/pemilu/hhcw...,ab2af948e91c08eda21644fc70f82fd5fe0856a2744b9d...,3471131004003,44.0,66.0,...,6.0,170.0,77.0,93.0,0.0,0.0,0.0,https://sirekap-obj-formc.kpu.go.id/d167/pemil...,https://sirekap-obj-formc.kpu.go.id/d167/pemil...,https://sirekap-obj-formc.kpu.go.id/d167/pemil...
2,2024-02-15 07:00:44,NaN,None,False,False,https://sirekap-obj-data.kpu.go.id/pemilu/hhcw...,a9226f2670040e29b156ff90847081788ea46ee21ac5f8...,3471131004009,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None
3,2024-02-25 14:00:00,hhcw,None,True,False,https://sirekap-obj-data.kpu.go.id/pemilu/hhcw...,17310bb21e055b36cadedaf3d430f4092f08466f689113...,3471131004006,104.0,96.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,https://sirekap-obj-formc.kpu.go.id/4aa9/pemil...,https://sirekap-obj-formc.kpu.go.id/4aa9/pemil...
4,2024-02-15 07:00:44,NaN,None,False,False,https://sirekap-obj-data.kpu.go.id/pemilu/hhcw...,64ac2f721d9d69f0543e31d72bcdf418f01d3c48ceafd4...,3471131004015,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None


## For Analysis

In [10]:
df_final_analysis = (
    df_final[["id", "scanned_page_1", "scanned_page_2", "scanned_page_3"]]
    .copy()
    .pipe(lambda frame: frame.melt(id_vars=["id"], value_vars=["scanned_page_1", "scanned_page_2", "scanned_page_3"]))
    .pipe(lambda frame: frame.assign(page=frame["variable"].str[-1].astype(int)))
    .rename(columns={"value": "url"})
    .drop(columns=["variable"])
    .dropna()
)

df_final_analysis.head()

,id,url,page
0,57b06ebd3928fa64149945243adfd8d231ffc71f26ea22...,https://sirekap-obj-formc.kpu.go.id/93b4/pemil...,1
1,ab2af948e91c08eda21644fc70f82fd5fe0856a2744b9d...,https://sirekap-obj-formc.kpu.go.id/d167/pemil...,1
6,20aa0e4d3543c3fce4e16529266a322e1d1a7c6d69b457...,https://sirekap-obj-formc.kpu.go.id/000d/pemil...,1
7,ac49a1a38bb40a032114568d36b3ff350963c5e4b520f8...,https://sirekap-obj-formc.kpu.go.id/de43/pemil...,1
8,d3f7599878e79dc3157679dfacc7817e974195b12c7eb9...,https://sirekap-obj-formc.kpu.go.id/98ee/pemil...,1


# Load

In [11]:
df_final.to_parquet("../data/clean/tps.parquet")

In [12]:
df_final_analysis.to_json("../data/clean/analyze-formc.jsonl", orient="records", lines=True)